In [1]:
import yaml
from pathlib import Path
import requests

In [24]:
with open("/home/zhouan/App/BenchmarkingClassifiers/snakemake/config/ground_truth_zymo_d6300.yml", 'r') as f:
    theoretical_abundance = {'species': yaml.load(f, Loader=yaml.FullLoader)}
    
tmp=list(theoretical_abundance['species'].keys())

s = requests.Session()
tmp

[287, 562, 28901, 1613, 1351, 1280, 1639, 1423, 4932, 5207]

In [ ]:
def fetch_fungus_or_nah(taxids):
    base_url = f'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/taxonomy/taxon/'
    taxids_str = ','.join(map(str, taxids))
    url = f'{base_url}{taxids_str}'
    params = {}
    try:
        response = s.get(url, params=params)
        response.raise_for_status() # 
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        r = response.json()
    return r
fetch_fungus_or_nah(tmp)

In [21]:
def fetch_reference_dataset_report(taxon, reference_only='true'):
    base_url = f'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/taxon/{taxon}/dataset_report?'
    params = {f'filters.reference_only': {reference_only},
              'filters.exclude_paired_reports': 'false',
              'filters.exclude_atypical': 'true'}
    try:
        response = s.get(base_url, params=params)
        # print(response.url)
        response.raise_for_status()
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        return response.json()



In [22]:
for taxid, seq_abund in theoretical_abundance['species'].items():
    r = fetch_reference_dataset_report(taxid)

r

{'reports': [{'accession': 'GCF_000091045.1',
   'current_accession': 'GCF_000091045.1',
   'paired_accession': 'GCA_000091045.1',
   'source_database': 'SOURCE_DATABASE_REFSEQ',
   'organism': {'tax_id': 214684,
    'organism_name': 'Cryptococcus neoformans var. neoformans JEC21',
    'infraspecific_names': {'strain': 'JEC21'}},
   'assembly_info': {'assembly_level': 'Chromosome',
    'assembly_status': 'current',
    'paired_assembly': {'accession': 'GCA_000091045.1',
     'status': 'current',
     'annotation_name': 'Annotation submitted by TIGR'},
    'assembly_name': 'ASM9104v1',
    'assembly_type': 'haploid',
    'bioproject_lineage': [{'bioprojects': [{'accession': 'PRJNA13856',
        'title': 'WGS sequencing of strain JEC21 (serotype D)'}]}],
    'bioproject_accession': 'PRJNA13856',
    'release_date': '2005-01-07',
    'submitter': 'TIGR',
    'refseq_category': 'reference genome',
    'biosample': {'accession': 'SAMN03081418',
     'last_updated': '2024-01-04T12:26:21.663

In [28]:
len(r['reports'])

1

In [35]:
def fetch_reference_sequence_report(accession):
    base_url = f'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/{accession}/sequence_reports?'
    params = {}
    try:
        response = s.get(base_url, params=params)
        response.raise_for_status()
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        return response.json()
r = fetch_reference_sequence_report('GCF_000091045.1')
r


{'reports': [{'assembly_accession': 'GCF_000091045.1',
   'chr_name': '1',
   'assigned_molecule_location_type': 'Chromosome',
   'refseq_accession': 'NC_006670.1',
   'assembly_unit': 'Primary Assembly',
   'length': 2300533,
   'genbank_accession': 'AE017341.1',
   'gc_count': '1117874',
   'gc_percent': 48.5,
   'role': 'assembled-molecule',
   'sequence_name': '1'},
  {'assembly_accession': 'GCF_000091045.1',
   'chr_name': '2',
   'assigned_molecule_location_type': 'Chromosome',
   'refseq_accession': 'NC_006684.1',
   'assembly_unit': 'Primary Assembly',
   'length': 1632307,
   'genbank_accession': 'AE017342.1',
   'gc_count': '790296',
   'gc_percent': 48.5,
   'role': 'assembled-molecule',
   'sequence_name': '2'},
  {'assembly_accession': 'GCF_000091045.1',
   'chr_name': '3',
   'assigned_molecule_location_type': 'Chromosome',
   'refseq_accession': 'NC_006685.1',
   'assembly_unit': 'Primary Assembly',
   'length': 2105742,
   'genbank_accession': 'AE017343.1',
   'gc_count

In [42]:
sequences = {}
lengths = 0
for sequence in r['reports']:
    if sequence['assigned_molecule_location_type'] == 'Plasmid':
        pass
    elif sequence['assigned_molecule_location_type'] == 'Chromosome':
        sequences[sequence['refseq_accession']] = sequence['length']
        lengths += sequence['length']
sequences


{'NC_006670.1': 2300533,
 'NC_006684.1': 1632307,
 'NC_006685.1': 2105742,
 'NC_006686.1': 1783081,
 'NC_006687.1': 1507550,
 'NC_006691.1': 1438950,
 'NC_006692.1': 1347793,
 'NC_006693.1': 1194300,
 'NC_006694.1': 1178688,
 'NC_006679.1': 1085720,
 'NC_006680.1': 1019846,
 'NC_006681.1': 906719,
 'NC_006682.1': 787999,
 'NC_006683.1': 762694}

In [41]:
lengths

19051922

In [7]:
import json
import logging
import math
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from rich.status import Status
import yaml
from pathlib import Path
theoretical_abundances = {}
data="/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers"
for ground_truth_file in ["ground_truth_tax_updated.yml" ,"ground_truth_updated.yml"]:
    ground_truth_file=Path(f"{data}/{ground_truth_file}")
    with open(ground_truth_file, 'r') as f:
        theoretical_abundance = {'species': yaml.load(f, Loader=yaml.FullLoader)}
    theoretical_abundance['genus'] = {}

    for k, v in theoretical_abundance['species'].items():
        genus_name = k.split()[0]
        if genus_name in theoretical_abundance['genus']:
            theoretical_abundance['genus'][genus_name] += v
        else:
            theoretical_abundance['genus'][genus_name] = v

    if ground_truth_file.stem.split('_')[-2] == 'tax':
        theoretical_abundances['tax'] = theoretical_abundance
    else:
        theoretical_abundances['sequence'] = theoretical_abundance
theoretical_abundances

{'tax': {'species': {'Pseudomonas aeruginosa': 0.067776,
   'Escherichia coli': 0.09147,
   'Salmonella enterica': 0.087407,
   'Limosilactobacillus fermentum': 0.205253,
   'Enterococcus faecalis': 0.147915,
   'Staphylococcus aureus': 0.150485,
   'Listeria monocytogenes': 0.144191,
   'Bacillus subtilis': 0.100715,
   'Saccharomyces cerevisiae': 0.002931,
   'Cryptococcus neoformans': 0.001857},
  'genus': {'Pseudomonas': 0.067776,
   'Escherichia': 0.09147,
   'Salmonella': 0.087407,
   'Limosilactobacillus': 0.205253,
   'Enterococcus': 0.147915,
   'Staphylococcus': 0.150485,
   'Listeria': 0.144191,
   'Bacillus': 0.100715,
   'Saccharomyces': 0.002931,
   'Cryptococcus': 0.001857}},
 'sequence': {'species': {'Pseudomonas aeruginosa': 0.12,
   'Escherichia coli': 0.12,
   'Salmonella enterica': 0.12,
   'Limosilactobacillus fermentum': 0.12,
   'Enterococcus faecalis': 0.12,
   'Staphylococcus aureus': 0.12,
   'Listeria monocytogenes': 0.12,
   'Bacillus subtilis': 0.12,
   'Sa

In [2]:
collected_metrics = {
    'Kraken2': {
        'TP': 10,
        'FP': 2,
        'FN': 3,
        'Precision': 0.833,
        'Recall': 0.769,
        'F1': 0.800
    },
    'MetaPhlAn': {
        'TP': 8,
        'FP': 1,
        'FN': 4,
        'Precision': 0.889,
        'Recall': 0.667,
        'F1': 0.762
    }
}
import pandas as pd

top_table = pd.DataFrame.from_dict(collected_metrics, orient='index').T
print(top_table)


           Kraken2  MetaPhlAn
TP          10.000      8.000
FP           2.000      1.000
FN           3.000      4.000
Precision    0.833      0.889
Recall       0.769      0.667
F1           0.800      0.762


In [3]:
pd.DataFrame.from_dict(collected_metrics, orient='index')

,TP,FP,FN,Precision,Recall,F1
Kraken2,10,2,3,0.833,0.769,0.800
MetaPhlAn,8,1,4,0.889,0.667,0.762


In [7]:
groups = ['Classified', 'Classified', 'Unclassified', 'Classified']
list_uc = ['no_hit',
           'ambigious',
           'unclassified',
           'missing_genus',
           'not_distributed',
           'rounding_error'
           ]
import pandas as pd

groups_cat = pd.Categorical(groups, categories=['Classified'] + list_uc)
print(groups_cat)

['Classified', 'Classified', NaN, 'Classified']
Categories (7, object): ['Classified', 'no_hit', 'ambigious', 'unclassified', 'missing_genus', 'not_distributed', 'rounding_error']


In [8]:
groups_cat.categories

Index(['Classified', 'no_hit', 'ambigious', 'unclassified', 'missing_genus',
       'not_distributed', 'rounding_error'],
      dtype='object')

In [12]:
from pathlib import Path
data="/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/output"
data=Path(data)
data.glob("/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/output/species" + '/' + '*.tsv')

<generator object Path.glob at 0x7fe004c8e500>

In [20]:
import itertools
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from dataclasses import dataclass
from sklearn import metrics
from typing import List
@dataclass # 通过 @dataclass，可以直接在类中定义属性，而不需要显式编写 __init__ 方法
class ClassificationResults:
    classifier: str
    results: pd.Series
    names_unclassified: List[str]

    def bundled_classified(self, normalized=False): #  将结果分为两类：已分类（Classified）和未分类（list_uc 中的单元）
        """
        'Classified' --> ground truth + classified but not in ground truth
        The rest are in list_uc and are as-is
        """
        groups = np.where(np.isin(self.results.index, list_uc), self.results.index, 'Classified') # 根据 self.results.index 中的分类单元是否在 list_uc 中，将分类单元分为两类：1. 在 list_uc 中的分类单元，保持原样；2. 不在 list_uc 中的分类单元，标记为 'Classified'；返回NumPy 数组
        groups_cat = pd.Categorical(groups,
                                    categories=['Classified'] + list_uc) # 将分类单元分为 'Classified'和 list_uc 中的类
        bundled_classified = self.results.groupby(groups_cat, observed=False).aggregate('sum') # 对'Classified'和 list_uc 的reads数量进行求和
        if normalized:
            return self._normalize_series(bundled_classified)
        return bundled_classified

In [21]:
list_uc = ['no_hit', 'ambigious', 'unclassified', 'missing_genus']
classifiers_input = [
    ClassificationResults(
        classifier='kraken2',
        results=pd.Series({
            'E_coli': 100,
            'S_aureus': 200,
            'B_subtilis': 50,
            'Unclassified': 30,
            'no_hit': 10,
            'ambigious': 5,
            'unclassified': 2,
            'missing_genus': 1
        }),
        names_unclassified=['no_hit', 'ambigious', 'unclassified', 'missing_genus']
    ),
    ClassificationResults(
        classifier='bracken',
        results=pd.Series({
            'E_coli': 150,
            'S_aureus': 180,
            'B_subtilis': 70,
            'Unclassified': 20,
            'no_hit': 10,
            'ambigious': 5,
            'unclassified': 2,
            'missing_genus': 1
        }),
        names_unclassified=['no_hit', 'ambigious', 'unclassified', 'missing_genus']
    )
]

In [27]:
un_classified = {}
list_abundance_classifiers = ['MetaPhlAn3', 'mOTUs2']
for classifier_result in classifiers_input:
    if classifier_result.classifier not in list_abundance_classifiers: # 如果分类器不直接输出丰度结果
        un_classified[classifier_result.classifier] = classifier_result.bundled_classified(False) # 返回一个字典，包含Classified'和 list_uc 分类的reads数量之和
un_classified_values = pd.DataFrame.from_dict(un_classified)
output_plots_organism="/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/output/species"
output_plots_organism = Path(output_plots_organism)
un_classified_values.to_csv(output_plots_organism / 'number_table.tsv', sep='\t')

In [28]:
enumerate(classifiers_input)